In [1]:
import numpy as np
import pandas as pd
import pickle
import sys
import os
import dsc
from dsc.query_engine import Query_Processor as dscQP
from dsc import dsc_io

import matplotlib.pyplot as plt
from pymir import mpl_stylesheet
from pymir import mpl_utils

In [2]:
def stratify_dfcol(df, colname, value):
    #return pd_utils.select_dfrows(df, [f"$({colname}) == {value}"])
    return df.loc[df[colname] == value]

def stratify_dfcols(df, condition_list):
    for (colname, value) in condition_list:
        df = stratify_dfcol(df, colname, value)
    return df

def stratify_dfcols_in_list(df, colname, values):
    return df.loc[df[colname].isin(values)]

In [3]:
dsc_output = "/home/saikatbanerjee/scratch/work/gradvi-experiments/linreg_indep"
dsc_fname  = os.path.basename(os.path.normpath(dsc_output))
db = os.path.join(dsc_output, dsc_fname + ".db")
elbopkl = os.path.join("../dsc/results", dsc_fname + "_elbo.pkl")

refresh_pickle = False

print(db)

/home/saikatbanerjee/scratch/work/gradvi-experiments/linreg_indep/linreg_indep.db


In [4]:
target = ["simulate", "simulate.sfix", "simulate.pve", "simulate.se", "simulate.dims", "fit"]
#condition = ["simulate.sfix == 2", "simulate.signal == 'normal'", "simulate.dims == '(50, 200)'"]
#groups = ["fit_cpt:"]
condition = [""]

In [6]:
qp = dscQP(db, target, condition)
qp_df = qp.output_table
df    = stratify_dfcols_in_list(qp_df, 'fit', ['gradvi_direct', 'gradvi_compound', 'mr_ash', 'mr_ash_lasso_init'])
df

,DSC,simulate,simulate.sfix,simulate.se:output,simulate.pve,simulate.output.file,simulate.dims,fit,fit.output.file
750,1,equicorrgauss,1,equicorrgauss/equicorrgauss_1,0.4,equicorrgauss/equicorrgauss_1,"(500,10000)",mr_ash,mr_ash/equicorrgauss_1_mr_ash_1
751,1,equicorrgauss,2,equicorrgauss/equicorrgauss_2,0.4,equicorrgauss/equicorrgauss_2,"(500,10000)",mr_ash,mr_ash/equicorrgauss_2_mr_ash_1
752,1,equicorrgauss,5,equicorrgauss/equicorrgauss_3,0.4,equicorrgauss/equicorrgauss_3,"(500,10000)",mr_ash,mr_ash/equicorrgauss_3_mr_ash_1
753,1,equicorrgauss,10,equicorrgauss/equicorrgauss_4,0.4,equicorrgauss/equicorrgauss_4,"(500,10000)",mr_ash,mr_ash/equicorrgauss_4_mr_ash_1
754,1,equicorrgauss,20,equicorrgauss/equicorrgauss_5,0.4,equicorrgauss/equicorrgauss_5,"(500,10000)",mr_ash,mr_ash/equicorrgauss_5_mr_ash_1
...,...,...,...,...,...,...,...,...,...
1345,10,equicorrgauss,1,equicorrgauss/equicorrgauss_146,0.8,equicorrgauss/equicorrgauss_146,"(500,10000)",gradvi_compound,gradvi_compound/equicorrgauss_146_gradvi_compo...
1346,10,equicorrgauss,2,equicorrgauss/equicorrgauss_147,0.8,equicorrgauss/equicorrgauss_147,"(500,10000)",gradvi_compound,gradvi_compound/equicorrgauss_147_gradvi_compo...
1347,10,equicorrgauss,5,equicorrgauss/equicorrgauss_148,0.8,equicorrgauss/equicorrgauss_148,"(500,10000)",gradvi_compound,gradvi_compound/equicorrgauss_148_gradvi_compo...
1348,10,equicorrgauss,10,equicorrgauss/equicorrgauss_149,0.8,equicorrgauss/equicorrgauss_149,"(500,10000)",gradvi_compound,gradvi_compound/equicorrgauss_149_gradvi_compo...


In [48]:
method = "gradvi_compound"
dfm = stratify_dfcol(df, "fit", method)
fprefix = dfm['fit.output.file'].values[0]
ext = 'pkl' if method.startswith('gradvi') else 'rds'
fname = os.path.join(dsc_output, f"{fprefix}.{ext}")

In [49]:
data = dsc_io.load_dsc(fname)

In [50]:
data['model'].keys()

dict_keys(['_dj', '_init_params', '_invert_method', '_invert_options', '_is_debug', '_is_elbo_calc', '_is_intercept', '_method', '_nclbk', '_objtype', '_opts', 'coef', 'elbo_path', 'fitobj', 'fun', 'grad', 'intercept', 'nfev', 'niter', 'njev', 'obj_path', 'prior', 'residual_var', 'success', 'theta'])

In [53]:
data['model']

{'_dj': array([500., 500., 500., ..., 500., 500., 500.]),
 '_init_params': (array([0., 0., 0., ..., 0., 0., 0.]),
  array([-2.99573207, -2.99573207, -2.99573207, -2.99573207, -2.99573207,
         -2.99573207, -2.99573207, -2.99573207, -2.99573207, -2.99573207,
         -2.99573207, -2.99573207, -2.99573207, -2.99573207, -2.99573207,
         -2.99573207, -2.99573207, -2.99573207, -2.99573207, -2.99573207]),
  0.059739301402223034),
 '_invert_method': None,
 '_invert_options': {},
 '_is_debug': False,
 '_is_elbo_calc': False,
 '_is_intercept': True,
 '_method': 'l-bfgs-b',
 '_nclbk': 763,
 '_objtype': 'reparametrize',
 '_opts': {'maxiter': 2000,
  'disp': False,
  'ftol': 1e-09,
  'gtol': 1e-09,
  'maxfun': 20000},
 'coef': array([-1.76255859e-05,  3.27542654e-06, -1.86067669e-06, ...,
        -4.49634793e-06,  1.05162121e-07,  1.08063857e-06]),
 'elbo_path': array([15194.65126735,  3587.53442001,   493.71348659,   481.25386122,
          477.4763341 ,   477.24240129,   476.53586436,  

In [44]:
data['model']['fit']['iter']

175

In [45]:
def get_convergence_from_result(data, method, mrash_maxiter = 2000):
    '''
    1: converged
    2: abnormal termination
    3: number of iterations reached limit
    4: unknown
    '''
    if method.startswith('gradvi'):
        success = data['model']['success']
        if success:
            conv_status = 1
        else:
            conv_message = data['model']['fitobj']['message']
            if "ITERATIONS REACHED LIMIT" in conv_message:
                conv_status = 3
            elif "ABNORMAL_TERMINATION_IN_LNSRCH" in conv_message:
                conv_status = 2
            else:
                conv_status = 4
    else:
        niter = data['model']['fit']['iter']
        if niter == mrash_maxiter:
            conv_status = 3
        else:
            conv_status = 1
    return conv_status

In [47]:
method = "gradvi_compound"
dfm = stratify_dfcol(df, "fit", method)

for i, dfrow in dfm.iterrows():
    fprefix = dfrow['fit.output.file']
    ext = 'pkl' if method.startswith('gradvi') else 'rds'
    fname = os.path.join(dsc_output, f"{fprefix}.{ext}")
    data = dsc_io.load_dsc(fname)
    conv_status = get_convergence_from_result(data, method)
    print(f"PVE {dfrow['simulate.pve']}, s = {dfrow['simulate.sfix']}\t{conv_status}")
    if conv_status != 1:
        #print(f"PVE {dfrow['simulate.pve']},s = {dfrow['simulate.sfix']}\t")
        print(fname)
        print(data['model']['fitobj']['message'])
        print(data['model']['fitobj']['success'])
        print(data['model']['elbo_path'][-1])
        print(data['model']['niter'])
        print(get_convergence_from_result(data, method))

PVE 0.4, s = 1	1
PVE 0.4, s = 2	1
PVE 0.4, s = 5	1
PVE 0.4, s = 10	1
PVE 0.4, s = 20	1
PVE 0.6, s = 1	1
PVE 0.6, s = 2	1
PVE 0.6, s = 5	1
PVE 0.6, s = 10	1
PVE 0.6, s = 20	1
PVE 0.8, s = 1	1
PVE 0.8, s = 2	1
PVE 0.8, s = 5	1
PVE 0.8, s = 10	1
PVE 0.8, s = 20	1
PVE 0.4, s = 1	1
PVE 0.4, s = 2	1
PVE 0.4, s = 5	1
PVE 0.4, s = 10	1
PVE 0.4, s = 20	1
PVE 0.6, s = 1	1
PVE 0.6, s = 2	1
PVE 0.6, s = 5	1
PVE 0.6, s = 10	1
PVE 0.6, s = 20	1
PVE 0.8, s = 1	1
PVE 0.8, s = 2	1
PVE 0.8, s = 5	1
PVE 0.8, s = 10	1
PVE 0.8, s = 20	1
PVE 0.4, s = 1	1
PVE 0.4, s = 2	1
PVE 0.4, s = 5	1
PVE 0.4, s = 10	1
PVE 0.4, s = 20	1
PVE 0.6, s = 1	1
PVE 0.6, s = 2	1
PVE 0.6, s = 5	1
PVE 0.6, s = 10	1
PVE 0.6, s = 20	1
PVE 0.8, s = 1	1
PVE 0.8, s = 2	1
PVE 0.8, s = 5	1
PVE 0.8, s = 10	1
PVE 0.8, s = 20	1
PVE 0.4, s = 1	1
PVE 0.4, s = 2	1
PVE 0.4, s = 5	1
PVE 0.4, s = 10	1
PVE 0.4, s = 20	1
PVE 0.6, s = 1	1
PVE 0.6, s = 2	1
PVE 0.6, s = 5	1
PVE 0.6, s = 10	1
PVE 0.6, s = 20	1
PVE 0.8, s = 1	1
PVE 0.8, s = 2	1
PVE 0.8, 

In [21]:
data['model'].keys()

dict_keys(['_dj', '_init_params', '_invert_method', '_invert_options', '_is_debug', '_is_elbo_calc', '_is_intercept', '_method', '_nclbk', '_objtype', '_opts', 'coef', 'elbo_path', 'fitobj', 'fun', 'grad', 'intercept', 'nfev', 'niter', 'njev', 'obj_path', 'prior', 'residual_var', 'success', 'theta'])

In [13]:
d

data['model']['fitobj']['message']

      fun: -30607.585319858154
 hess_inv: <10021x10021 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 2.97422810e-03,  1.53237852e-04, -1.47324033e-05, ...,
        1.42479622e-03,  1.77491234e-03,  3.65539389e-01])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 822
      nit: 763
     njev: 822
   status: 0
  success: True
        x: array([-0.04102311,  0.01922643, -0.01265711, ..., -3.28205805,
       -3.87436708,  0.32630455])

In [ ]:
    dfm = stratify_dfcol(df, "fit", method)
    assert(dfm.shape[0] == 1)
    fprefix = dfm['fit.output.file'].values[0]
    ext = 'pkl' if method.startswith('gradvi') else 'rds'
    fname = os.path.join(dsc_output, f"{fprefix}.{ext}")
    data = dsc_io.load_dsc(fname)
    if method.startswith('gradvi'):
        return data['model']['elbo_path'][-1]
    else:
        return data['model']['fit']['varobj'][-1]